<h1> Scraping Postal Codes Of Torronto City </h1>

<h4>Importing dependencies</h4>

In [32]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

<h4>Requesting source page</h4>

In [100]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

<h4>Using BeautifulSoup for scraping the data</h4>

In [ ]:
soup = BeautifulSoup(source, 'lxml')
table = soup.find('tbody')
postcode = []
borough = []
neighborhood = []
for row in table.findAll('tr'):
    temp = []
    for each_data in row.findAll('td'):
        temp.append(each_data.text)
    if len(temp) and temp[1] != 'Not assigned': #Ignoring cells with a borough that is Not assigned
        postcode.append(temp[0])
        borough.append(temp[1])
        temp_2 = temp[2].rstrip('\n')
        if temp_2 == 'Not assigned': #If a cell has a Not assigned neighborhood
            temp2 = temp[1]          #then the neighborhood will be the same as the borough
        neighborhood.append(temp_2)
data = {'PostalCode': postcode, 'Borough': borough, 'Neighborhood': neighborhood} #The dataframe will consist of three columns:
                                                                                  #PostalCode, Borough, and Neighborhood

<h4>Storing the scraped data in pandas dataframe</h4>

In [101]:
df = pd.DataFrame(data)                                                           
df = df.groupby("PostalCode").agg(lambda x:','.join(set(x)))

<h4>The number of rows of your dataframe

In [104]:
df.shape

(103, 2)

<h4>Displaying any 10 data from the dataframe</h4>

In [102]:
df.sample(10)

,Borough,Neighborhood
PostalCode,,
M5N,Central Toronto,Roselawn
M3J,North York,"Northwood Park,York University"
M5K,Downtown Toronto,"Toronto Dominion Centre,Design Exchange"
M9L,North York,Humber Summit
M5X,Downtown Toronto,"First Canadian Place,Underground city"
M2H,North York,Hillcrest Village
M4V,Central Toronto,"South Hill,Deer Park,Forest Hill SE,Rathnelly,..."
M9R,Etobicoke,"Martin Grove Gardens,Kingsview Village,St. Phi..."
M7R,Mississauga,Canada Post Gateway Processing Centre
